In [8]:
import os
import pandas as pd
import geopandas as gpd
import xarray as xr
import math

**Build ML models to predict fire ignition for a pixel**

Features: FWI, lat, lon, elevation, landcover (2010), distance, month
Label: ignited or not ignited
Model: XGBoost
Optimization Method: Beyas optimization using Optuna
Data range: EU

Steps:
1. Merge positive and negative labels of fire intensity and FWi data
    1.1 Load left join data (fire intensity left join FWI, resolution 1km) and count how many data points it has
    1.2 Assign positive labels (ignited) to the data points with distance <= 31 * sqrt(2) / 2 and only keep rows with positive labels
    1.3 Load right join data (fire intensity right join FWI, resolution 31km) and count how may data points it has
    1.4 Down sampling the data to match the number of left join data
    1.5 Assign negative labels (ignited) to the data points with distance > 31 * sqrt(2) / 2 and only keep rows with negative labels
    1.6 Merge data and save the geo dataframe and save the dataframe
2. Add land cover (aggregated) data to pixel according to lon and lat
    2.1 Load land cover shapefile and convert to geopandas dataframe
    2.2 for each pixel (1x1 for left and 31x31 for right), calculate the average probability of fire ignition according to Clamada Master thesis Table 2.1
3. Add elevation data to pixel according to lon and lat
    3.1 Load elevation shapefile and convert to geopandas dataframe
    3.2 for each pixel (1x1 for left and 31x31 for right), calculate the average elevation
4. Build ML models to predict whether a pixel is ignited
5. Tune hyperparameters

# 1. Merge positive and negative labels of fire intensity and FWi data

In [24]:
from geopy.distance import great_circle

# Define a function to calculate the distance
def calculate_distance(row):
    coords_1 = (row['latitude_left'], row['longitude_left'])
    coords_2 = (row['latitude_right'], row['longitude_right'])
    return great_circle(coords_1, coords_2).kilometers

folder = '../../climada_petals/data/wildfire/output/2013/'

## 1.1 Load left join data (fire intensity left join FWI, resolution 1km)

In [25]:
merged_eu_2013_left_gdf_filename = 'merged_eu_2013_left_gdf'
df_left_join = gpd.read_file(os.path.join(folder, merged_eu_2013_left_gdf_filename))
df_left_join.shape

(78382, 17)

In [26]:
# Apply the function to each row in the GeoDataFrame and create a new column 'distance_km'
df_left_join['distance_km'] = df_left_join.apply(calculate_distance, axis=1)

In [27]:
df_left_join['distance_km'].describe()

count    78382.000000
mean         9.123286
std          3.588911
min          0.035163
25%          6.526173
50%          9.238830
75%         12.049334
max         17.695499
Name: distance_km, dtype: float64

In [28]:
df_left_join

,latitude_left,longitude_left,brightness,satellite,instrument,confidence,bright_t31,frp,daynight,index,latitude_right,longitude_right,surface,fwi,distance,date,geometry,distance_km
0,35.8073,-0.2538,310.2,Aqua,MODIS,79,277.6,25.0,N,313919,35.75,-0.25,0.0,6.847656,0.057426,2013-01-01,POINT (-0.25380 35.80730),6.380693
1,47.8587,33.4466,308.0,Terra,MODIS,63,272.6,27.4,D,243494,47.75,33.50,0.0,0.214844,0.121108,2013-01-01,POINT (33.44660 47.85870),12.727889
2,49.6728,18.6611,307.4,Terra,MODIS,57,276.5,12.8,D,231915,49.75,18.75,0.0,1.417969,0.117741,2013-01-01,POINT (18.66110 49.67280),10.702774
3,37.3934,39.4902,303.1,Aqua,MODIS,55,287.0,14.4,D,302558,37.50,39.50,0.0,3.445312,0.107050,2013-01-01,POINT (39.49020 37.39340),11.884926
4,36.8729,6.9397,320.8,Terra,MODIS,75,286.9,32.9,D,306748,36.75,7.00,0.0,3.714844,0.136896,2013-01-01,POINT (6.93970 36.87290),14.682411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78377,52.1556,10.4051,307.4,Aqua,MODIS,65,276.4,10.9,D,378136842,52.25,10.50,0.0,0.230469,0.133856,2013-12-31,POINT (10.40510 52.15560),12.329164
78378,52.1535,10.3907,300.4,Aqua,MODIS,19,276.3,6.7,D,378136842,52.25,10.50,0.0,0.230469,0.145804,2013-12-31,POINT (10.39070 52.15350),13.062298
78379,35.8027,-0.2461,310.6,Aqua,MODIS,29,283.8,43.8,D,378233279,35.75,-0.25,0.0,0.609375,0.052844,2013-12-31,POINT (-0.24610 35.80270),5.870533
78380,54.5777,-1.1428,310.4,Terra,MODIS,71,275.8,15.6,N,378125275,54.50,-1.25,0.0,0.085938,0.132398,2013-12-31,POINT (-1.14280 54.57770),11.066649


In [30]:
# only keep the relevant rows
df_left_join = df_left_join[['latitude_left', 'longitude_left', 'brightness', 'confidence', 'bright_t31', 'fwi', 'distance_km', 'geometry']]

## 1.2 Assign positive labels (ignited) to the data points with distance <= 31 * sqrt(2) / 2 and only keep rows with positive labels

In [31]:
# Drop rows with distance_km > 31 km
df_left_join = df_left_join[df_left_join['distance_km'] <= 31 * math.sqrt(2) / 2 ]
# Drop rows with confidence < 30
df_left_join = df_left_join[df_left_join['confidence'] >= 30]

In [32]:
df_left_join.shape

(73505, 8)

In [50]:
df_left_join['ignited'] = True

## 1.3 Load and down sampling right join data (fire intensity right join FWI, resolution 31km)

In [13]:
folder = '../../climada_petals/data/wildfire/output/2013/'
# only NetCDF works for interpolated data
file = 'merged_eu_2013_right_gdf'
df = gpd.read_file(os.path.join(folder, file))

df.shape

(15210699, 17)

## 1.4 Down sampling the data to match the number of left join data

In [41]:
# Randomly sample 100,000 rows without replacement
df_right_join = df.sample(n=100000, replace=False)

In [42]:
# Apply the function to each row in the GeoDataFrame and create a new column 'distance_km'
df_right_join['distance_km'] = df_right_join.apply(calculate_distance, axis=1)

In [16]:
df_right_join['distance_km'].describe()

count    100000.000000
mean       1828.264519
std        1066.731837
min           0.170296
25%         910.516072
50%        1813.205400
75%        2691.196643
max        4886.774212
Name: distance_km, dtype: float64

In [17]:
df_right_join

,latitude_left,longitude_left,brightness,satellite,instrument,confidence,bright_t31,frp,daynight,index,latitude_right,longitude_right,surface,fwi,distance,date,geometry,distance_km
7997239,61.7665,-120.8940,317.0,Aqua,MODIS,94,285.6,23.3,N,209866558,65.50,-120.50,0.0,4.015625,3.754232,2013-07-22,POINT (-120.50000 65.50000),415.600420
1918428,56.5077,-115.2732,314.3,Aqua,MODIS,0,272.1,18.2,D,62379063,75.50,-74.25,0.0,NaN,45.206309,2013-03-02,POINT (-74.25000 75.50000),2698.453412
8040262,50.3483,-66.3058,301.0,Terra,MODIS,22,285.8,11.0,D,210880501,69.75,-74.75,0.0,3.699219,21.159643,2013-07-23,POINT (-74.75000 69.75000),2202.833648
15028115,54.2980,-115.2340,313.1,Terra,MODIS,0,265.8,33.2,D,363477236,74.00,-91.00,0.0,0.671875,31.232284,2013-12-17,POINT (-91.00000 74.00000),2444.590971
11554887,50.9914,-98.4281,300.6,Terra,MODIS,31,277.8,31.6,N,285606381,74.50,-84.75,0.0,0.699219,27.198248,2013-10-03,POINT (-84.75000 74.50000),2689.217168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12716780,58.3739,-115.5485,305.4,Aqua,MODIS,22,267.0,11.6,N,310476479,82.75,-120.25,0.0,NaN,24.825357,2013-10-27,POINT (-120.25000 82.75000),2713.937588
14641135,49.0483,-124.0786,313.8,Aqua,MODIS,0,265.6,16.9,D,353112124,71.00,-89.00,0.0,0.316406,41.380978,2013-12-07,POINT (-89.00000 71.00000),3033.905276
13632324,49.8492,-115.6591,322.0,Aqua,MODIS,53,281.6,47.1,D,329306115,58.25,-71.25,0.0,0.183594,45.196699,2013-11-14,POINT (-71.25000 58.25000),2983.328388
13043849,54.6503,-106.4851,305.0,Terra,MODIS,53,274.8,24.7,D,316841336,59.25,-106.00,0.0,0.207031,4.625209,2013-11-02,POINT (-106.00000 59.25000),512.305651


In [43]:
# only keep the relevant rows
df_right_join = df_right_join[['latitude_right', 'longitude_right', 'brightness', 'confidence', 'bright_t31', 'fwi', 'distance_km', 'geometry']]

In [46]:
df_right_join.to_file(os.path.join(folder, f'merged_eu_2013_right_downsampled_gdf'), driver='GPKG')

In [ ]:
df_right_join = gpd.read_file(os.path.join(folder, 'merged_eu_2013_right_downsampled_gdf'))

## 1.5 Assign negative labels (ignited) to the data points with distance > 31 * sqrt(2) / 2 and only keep rows with negative labels

In [44]:
# Drop rows with distance_km > 31 km
df_right_join = df_right_join[df_right_join['distance_km'] > 31 * math.sqrt(2) / 2 ]
# Drop rows with confidence < 30
df_right_join = df_right_join[df_right_join['confidence'] >= 30]

In [45]:
df_right_join.shape

(77314, 8)

In [48]:
df_right_join['ignited'] = False

## 1.6 Merge data and save the geo dataframe and save the dataframe

In [58]:
df_right_join

,latitude_right,longitude_right,brightness,confidence,bright_t31,fwi,distance_km,geometry,ignited
249533,79.75,-54.25,305.6,62,267.2,0.273438,3520.446998,POINT (-54.25000 79.75000),False
7789791,73.75,-128.25,304.3,60,283.7,NaN,1339.569774,POINT (-128.25000 73.75000),False
2595121,81.00,-119.00,323.8,62,272.7,NaN,2710.101566,POINT (-119.00000 81.00000),False
1047826,68.50,-133.25,308.5,63,270.9,0.113281,1540.044702,POINT (-133.25000 68.50000),False
2752567,73.50,-116.75,308.4,74,268.2,0.492188,1885.285524,POINT (-116.75000 73.50000),False
...,...,...,...,...,...,...,...,...,...
12693671,64.75,-96.25,305.7,62,271.0,0.425781,1529.782090,POINT (-96.25000 64.75000),False
6937636,54.00,-127.25,324.6,72,301.6,0.500000,696.592985,POINT (-127.25000 54.00000),False
7178516,56.75,-61.75,320.5,56,290.8,0.003906,513.893917,POINT (-61.75000 56.75000),False
9944033,72.25,-102.25,322.1,81,281.8,0.289062,1687.857051,POINT (-102.25000 72.25000),False


In [59]:
# Rename the columns to match
df_left_join = df_left_join.rename(columns={'latitude_left': 'latitude', 'longitude_left': 'longitude'})
df_right_join = df_right_join.rename(columns={'latitude_right': 'latitude', 'longitude_right': 'longitude'})

# Concatenate the GeoDataFrames
gdf_concat = pd.concat([df_left_join, df_right_join], ignore_index=True)

# Ensure the concatenated DataFrame is still a GeoDataFrame
gdf_concat = gpd.GeoDataFrame(gdf_concat, geometry='geometry')
gdf_concat

,latitude,longitude,brightness,confidence,bright_t31,fwi,distance_km,geometry,ignited
0,35.8073,-0.2538,310.2,79,277.6,6.847656,6.380693,POINT (-0.25380 35.80730),True
1,47.8587,33.4466,308.0,63,272.6,0.214844,12.727889,POINT (33.44660 47.85870),True
2,49.6728,18.6611,307.4,57,276.5,1.417969,10.702774,POINT (18.66110 49.67280),True
3,37.3934,39.4902,303.1,55,287.0,3.445312,11.884926,POINT (39.49020 37.39340),True
4,36.8729,6.9397,320.8,75,286.9,3.714844,14.682411,POINT (6.93970 36.87290),True
...,...,...,...,...,...,...,...,...,...
150814,64.7500,-96.2500,305.7,62,271.0,0.425781,1529.782090,POINT (-96.25000 64.75000),False
150815,54.0000,-127.2500,324.6,72,301.6,0.500000,696.592985,POINT (-127.25000 54.00000),False
150816,56.7500,-61.7500,320.5,56,290.8,0.003906,513.893917,POINT (-61.75000 56.75000),False
150817,72.2500,-102.2500,322.1,81,281.8,0.289062,1687.857051,POINT (-102.25000 72.25000),False


In [60]:
gdf_concat.to_file(os.path.join(folder, f'merged_eu_2013_gdf'), driver='GPKG')

# 2. Add land cover (aggregated) data to pixel according to lon and lat

## 2.1 Load land cover shapefile and convert to geopandas dataframe

In [ ]:
merged_eu_2013_gdf = gpd.read_file(os.path.join(folder, f'merged_eu_2013_gdf'))

## 2.2 for each pixel (1x1 for left and 31x31 for right), calculate the average probability of fire ignition according to Clamada Master thesis Table 2.1


# 3. Add elevation data to pixel according to lon and lat
## 3.1 Load elevation shapefile and convert to geopandas dataframe

Create a Virtual Raster (VRT)
You can use the GDAL command-line tools or rasterio's virtual raster capabilities in Python to combine your GeoTIFF files into a VRT.
Using GDAL Command Line:
gdalbuildvrt combined_elevation.vrt 50N000E_20101117_gmted_mea075.tif 50N030W_20101117_gmted_mea075.tif 50N030E_20101117_gmted_mea075.tif 30N030W_20101117_gmted_mea075.tif 30N000E_20101117_gmted_mea075.tif 30N030E_20101117_gmted_mea075.tif

In [71]:
import rasterio
from rasterio.merge import merge
from rasterio.io import MemoryFile

elevation_folder = '../../climada_petals/data/wildfire/elevation/'
# List of your GeoTIFF files
files = ['50N000E_20101117_gmted_mea075.tif', '50N030W_20101117_gmted_mea075.tif', '50N030E_20101117_gmted_mea075.tif', '30N030W_20101117_gmted_mea075.tif', '30N000E_20101117_gmted_mea075.tif', '30N030E_20101117_gmted_mea075.tif']


# Open all files
src_files = [rasterio.open(os.path.join(elevation_folder, f)) for f in files]

# Create a virtual mosaic (in-memory, no VRT file written)
mosaic, out_transform = merge(src_files)

# Create a Virtual Raster (VRT) file in memory (alternative to writing to disk)
with MemoryFile() as memfile:
    with memfile.open(driver='GTiff', height=mosaic.shape[1], width=mosaic.shape[2],
                      count=1, dtype=mosaic.dtype, transform=out_transform,
                      crs=src_files[0].crs) as dataset:
        dataset.write(mosaic[0], 1)
        virtual_raster = dataset.name  # Path to the in-memory dataset

## 3.2 for each pixel (1x1 for left and 31x31 for right), calculate the average elevation

In [62]:
merged_eu_2013_gdf = gdf_concat

In [73]:
# Load the virtual raster
elevation_raster = rasterio.open(os.path.join(elevation_folder, 'combined_elevation.vrt'))
# Ensure CRS alignment
if elevation_raster.crs != merged_eu_2013_gdf.crs:
    elevation_gdf = elevation_raster.to_crs(merged_eu_2013_gdf.crs)

In [74]:
# Function to get elevation data from raster
def get_elevation(lon, lat, raster):
    for val in raster.sample([(lon, lat)]):
        return val[0]  # Assuming elevation data is in the first band

# Apply the function to each row in the geodataframe
merged_eu_2013_gdf['elevation'] = merged_eu_2013_gdf.apply(
    lambda row: get_elevation(row.geometry.x, row.geometry.y, elevation_raster), axis=1
)

# 4. Build ML models to predict whether a pixel is ignited